In [15]:
# Tratamiento de datos
# ==============================================================================
import pandas as pd
import numpy as np
import random

# Preprocesamiento
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Métricas
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

## XGBoost
from xgboost import XGBRegressor

from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.model_selection import PredefinedSplit

# Ajuste de hiperparámetros 
from optuna.integration import OptunaSearchCV
from optuna.distributions import IntDistribution, FloatDistribution
from optuna.logging import set_verbosity, WARNING
set_verbosity(WARNING)

# Semilla
random_seed = 36
np.random.seed(random_seed)
random.seed(random_seed)

# Guardado del modelo
import pickle

## Carga de los datos

In [16]:
disp_df = pd.read_csv("disp_st36ns1.txt.bz2", compression="bz2", index_col=0)
comp_df = pd.read_csv("comp_st36ns1.txt.bz2", compression="bz2", index_col=0)

Se añade el día del año al dataset.

Suponemos que los datos comienzan el 01 de enero y continúan 12 años

In [17]:
daylist = []
for year in range(12):
    for day in range(365):
        daylist.append(day)
disp_df['date'] = daylist
# Carga 2 años en días a los datos de competición. Comprobado que sigue la misma distribución que disp
comp_df['date'] = daylist[0:733]

Se transforman los índices de tipo texto a tipo entero

In [18]:
disp_df = disp_df.reset_index(drop=True)
Y_df = disp_df['salida']
X_df = disp_df.drop(columns='salida')

Usamos el split pedido. Por otra parte hicimos un split con un k-fold de 5 y obtuvimos un R^2 un 2% superior. Dejamos el actual por ser el que nos piden

In [19]:
X_train, X_test = np.array_split(X_df, [3650])
y_train, y_test = np.array_split(Y_df, [3650])
# The indices which have the value -1 will be kept in train.
train_indices = np.full((2920,), -1, dtype=int) # 8 years
# The indices which have zero or positive values, will be kept in validation
val_indices = np.full((730,), 0, dtype=int) # 2 years
test_fold = np.append(train_indices, val_indices)

ps = PredefinedSplit(test_fold)

In [20]:
def evaluate(model, X, y, cv=5, fit_params=None, tune=False, metric="neg_mean_absolute_error", jobs=1):
    """
     Evalúa cualquier pipeline por validación cruzada.
     Tiene la capacidad de hace una estimación de hiperparámetros con algoritmos heurísticos.
     Tiempo máximo de ejecución 15 minutos en caso de ajuste
    """
    np.random.seed(random_seed)
    random.seed(random_seed)
    optuna_search = OptunaSearchCV(estimator=model, 
                                        param_distributions=fit_params, 
                                        cv=cv,
                                        n_trials=500,
                                        timeout=900,
                                        scoring=metric,
                                        random_state=random_seed,
                                        verbose=0,
                                        n_jobs=jobs)
    optuna_search.fit(X, y)

    return optuna_search

# Entrenamiento de XGBoost

In [21]:
fit_params={
    'xgboost__learning_rate': FloatDistribution(0, 1),
    'xgboost__gamma': FloatDistribution(10e-3, 10e5, log=True),
    'xgboost__max_depth': IntDistribution(1, 10),
    'xgboost__min_child_weight': IntDistribution(0, 10),
    'xgboost__subsample': FloatDistribution(0, 1),
    'xgboost__lambda': FloatDistribution(10e-3, 10e5, log=True),
    'xgboost__alpha': FloatDistribution(10e-3, 10e3, log=True),
    'xgboost__colsample_bytree': FloatDistribution(0, 1)
}

Para nuestras propias pruebas usamos un k-fold de 5 . Realmente los datos no son una serie temporal (los datos del día anterior no afectan al día nuevo)

Hacemos esto para mejorar en un 2% el R^2 respecto al modelo con el split original

In [22]:
np.random.seed(random_seed)
random.seed(random_seed)
pipe_xgboost = Pipeline([
    ('scaler', StandardScaler()),
    ('xgboost', XGBRegressor(eval_metric='mae', seed=random_seed))
])
xgboost_mae_tuned = evaluate(pipe_xgboost, X_train, y_train, cv=ps, fit_params=fit_params, tune=True, metric="neg_mean_absolute_error", jobs=-1)
xgboost_mae_tuned.best_estimator_

/var/folders/2b/sn9pm__s4xq2pw_dts6nk3y00000gn/T/ipykernel_2059/1456660592.py:9: ExperimentalWarning: OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.
  optuna_search = OptunaSearchCV(estimator=model,


Pipeline(steps=[('scaler', StandardScaler()),
                ('xgboost',
                 XGBRegressor(alpha=41.13647813861622, base_score=None,
                              booster=None, callbacks=None,
                              colsample_bylevel=None, colsample_bynode=None,
                              colsample_bytree=0.8591015119674175,
                              early_stopping_rounds=None,
                              enable_categorical=False, eval_metric='mae',
                              feature_types=None, gamma=44.33043896720049,
                              gpu_id=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None,
                              lambda=48.05729434719377,
                              learning_rate=0.10943188604519347, max_bin=None,
                              max_cat_threshold=None, max_cat_to_onehot=None,
                              max_delta_step=None, max_depth=6, max_leaves=None,
                              min_child_weight=8, missing=nan,
                              monotone_constraints=None, n_estimators=100,
                              n_jobs=None, num_parallel_tree=None, ...))])

In [23]:
xgboost_mae_tuned.best_estimator_.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('xgboost',
                 XGBRegressor(alpha=41.13647813861622, base_score=None,
                              booster=None, callbacks=None,
                              colsample_bylevel=None, colsample_bynode=None,
                              colsample_bytree=0.8591015119674175,
                              early_stopping_rounds=None,
                              enable_categorical=False, eval_metric='mae',
                              feature_types=None, gamma=44.33043896720049,
                              gpu_id=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None,
                              lambda=48.05729434719377,
                              learning_rate=0.10943188604519347, max_bin=None,
                              max_cat_threshold=None, max_cat_to_onehot=None,
                              max_delta_step=None, max_depth=6, max_leaves=None,
                              min_child_weight=8, missing=nan,
                              monotone_constraints=None, n_estimators=100,
                              n_jobs=None, num_parallel_tree=None, ...))])

In [24]:
print("MAE de test de XGBoost", mean_absolute_error(y_test, xgboost_mae_tuned.best_estimator_.predict(X_test)))
print("RMSE de test de XGBoost", np.sqrt(mean_squared_error(y_test, xgboost_mae_tuned.best_estimator_.predict(X_test))))
print("R2 de test de XGBoost", metrics.r2_score(y_test, xgboost_mae_tuned.best_estimator_.predict(X_test)))

MAE de test de XGBoost 2112332.0941780824
RMSE de test de XGBoost 3190582.7717072302
R2 de test de XGBoost 0.8229984761118072


Modelo final

In [25]:
xgboost_mae_tuned.best_estimator_.fit(X_df, Y_df)

Pipeline(steps=[('scaler', StandardScaler()),
                ('xgboost',
                 XGBRegressor(alpha=41.13647813861622, base_score=None,
                              booster=None, callbacks=None,
                              colsample_bylevel=None, colsample_bynode=None,
                              colsample_bytree=0.8591015119674175,
                              early_stopping_rounds=None,
                              enable_categorical=False, eval_metric='mae',
                              feature_types=None, gamma=44.33043896720049,
                              gpu_id=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None,
                              lambda=48.05729434719377,
                              learning_rate=0.10943188604519347, max_bin=None,
                              max_cat_threshold=None, max_cat_to_onehot=None,
                              max_delta_step=None, max_depth=6, max_leaves=None,
                              min_child_weight=8, missing=nan,
                              monotone_constraints=None, n_estimators=100,
                              n_jobs=None, num_parallel_tree=None, ...))])

In [26]:
predictions = xgboost_mae_tuned.best_estimator_.predict(comp_df)

Guardado de las predicciones

In [27]:
np.savetxt("predicciones.csv", predictions)

Guardado del modelo

In [28]:
pickle.dump(xgboost_mae_tuned.best_estimator_, open("modelo_final.pkl", "wb"))